<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EA%B3%BC%EC%A0%9C/poly_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 5기분들 코드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 5기 분들 git파일 다운로드
!git clone https://github.com/doh0106/Poly-Encoder.git

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 368, done.
remote: Counting objects: 100% (360/360), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 368 (delta 197), reused 291 (delta 138), pack-reused 8
Receiving objects: 100% (368/368), 83.34 MiB | 24.05 MiB/s, done.
Resolving deltas: 100% (199/199), done.


#### 데이터 생성

In [2]:
# 데이터 불러오기
import pandas as pd

excel = "/content/data_sample.xlsx"
df = pd.read_excel(excel)

In [67]:
# 랜덤으로 샘플데이터 생성
df1 = df.sample(n=2000, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)
df1

,표제어,뜻풀이
1310,딸리다,어떤 것에 매이거나 붙어 있다.
1793,보편화,사회에 널리 퍼짐. 또는 그렇게 되게 함.
4486,헷갈리다,정신이 어지럽고 혼란스럽게 되다.
3312,자재,어떤 것을 만들 때 필요한 기본적인 물건이나 재료.
1242,뒤덮이다,빈 곳이 없게 모두 덮이다.
...,...,...
132,강우량,일정 기간 동안 일정한 곳에 내린 비의 양.
3787,쪼그리다,누르거나 구겨서 부피를 작게 만들다.
2110,상차림,상에 음식을 마련해 놓는 일. 또는 그렇게 마련해 놓은 상.
23,가리다,막거나 감추어 보이거나 통하지 않게 되다.


In [68]:
# 표제어 가나다 순으로 정렬(오름차순)
sorted_df = df1.sort_values(by=['표제어', '뜻풀이'])
sorted_df

,표제어,뜻풀이
0,가,어떤 장소나 물건의 둘레나 끝부분.
4,가구,한 집에서 함께 사는 사람들의 집단.
6,가까스로,매우 어렵게 힘을 들여.
7,가냘프다,"몸, 팔, 다리 등이 가늘고 연약하다."
8,가누다,몸이나 몸의 일부를 바르게 하다.
...,...,...
4693,희소식,기쁜 소식.
4697,힐끗,슬쩍 한 번 흘겨보는 모양.
4698,힘겹다,힘이 모자라거나 부족하여 어떤 일을 당해 내기 어렵다.
4699,힘없다,기운이나 의욕 등이 없다.


In [69]:
# 원본데이터와 샘플데이터가 동일한지 확인
found_word1 = sorted_df.loc[sorted_df['표제어'] == '타이르다']
found_word2 = sorted_df.loc[df1['표제어'] == '타이르다']
# 결과 출력
print(found_word1, found_word2)

Empty DataFrame
Columns: [표제어, 뜻풀이]
Index: [] Empty DataFrame
Columns: [표제어, 뜻풀이]
Index: []


In [70]:
# 데이터 질문 추가
sorted_df['Q1'] = sorted_df['표제어'].apply(lambda x: f"'{x}'의 뜻은?")
sorted_df['Q2'] = sorted_df['표제어'].apply(lambda x: f"'{x}'의 의미를 알려줘?")
sorted_df['Q3'] = sorted_df['표제어'].apply(lambda x: f"'{x}'의 예시가 뭐야?")
sorted_df['Q4'] = sorted_df['표제어'].apply(lambda x: f"'{x}'의 예문을 들어줘")
sorted_df

,표제어,뜻풀이,Q1,Q2,Q3,Q4
0,가,어떤 장소나 물건의 둘레나 끝부분.,'가'의 뜻은?,'가'의 의미를 알려줘?,'가'의 예시가 뭐야?,'가'의 예문을 들어줘
4,가구,한 집에서 함께 사는 사람들의 집단.,'가구'의 뜻은?,'가구'의 의미를 알려줘?,'가구'의 예시가 뭐야?,'가구'의 예문을 들어줘
6,가까스로,매우 어렵게 힘을 들여.,'가까스로'의 뜻은?,'가까스로'의 의미를 알려줘?,'가까스로'의 예시가 뭐야?,'가까스로'의 예문을 들어줘
7,가냘프다,"몸, 팔, 다리 등이 가늘고 연약하다.",'가냘프다'의 뜻은?,'가냘프다'의 의미를 알려줘?,'가냘프다'의 예시가 뭐야?,'가냘프다'의 예문을 들어줘
8,가누다,몸이나 몸의 일부를 바르게 하다.,'가누다'의 뜻은?,'가누다'의 의미를 알려줘?,'가누다'의 예시가 뭐야?,'가누다'의 예문을 들어줘
...,...,...,...,...,...,...
4693,희소식,기쁜 소식.,'희소식'의 뜻은?,'희소식'의 의미를 알려줘?,'희소식'의 예시가 뭐야?,'희소식'의 예문을 들어줘
4697,힐끗,슬쩍 한 번 흘겨보는 모양.,'힐끗'의 뜻은?,'힐끗'의 의미를 알려줘?,'힐끗'의 예시가 뭐야?,'힐끗'의 예문을 들어줘
4698,힘겹다,힘이 모자라거나 부족하여 어떤 일을 당해 내기 어렵다.,'힘겹다'의 뜻은?,'힘겹다'의 의미를 알려줘?,'힘겹다'의 예시가 뭐야?,'힘겹다'의 예문을 들어줘
4699,힘없다,기운이나 의욕 등이 없다.,'힘없다'의 뜻은?,'힘없다'의 의미를 알려줘?,'힘없다'의 예시가 뭐야?,'힘없다'의 예문을 들어줘


In [71]:
# '표제어' 열 삭제
sorted_df = sorted_df.drop("표제어", axis=1)

# 인덱스 열 추가
sorted_df['index'] = range(1, len(sorted_df) + 1)

# 열 순서를 'Q1', 'Q2', 'Q3', 'Q4', '뜻풀이' 순서로 변경
sorted_df.rename(columns={"뜻풀이": "answer"}, inplace=True)
sorted_df = sorted_df[["index", "Q1", "Q2", "Q3", "Q4", "answer"]]

sorted_df

,index,Q1,Q2,Q3,Q4,answer
0,1,'가'의 뜻은?,'가'의 의미를 알려줘?,'가'의 예시가 뭐야?,'가'의 예문을 들어줘,어떤 장소나 물건의 둘레나 끝부분.
4,2,'가구'의 뜻은?,'가구'의 의미를 알려줘?,'가구'의 예시가 뭐야?,'가구'의 예문을 들어줘,한 집에서 함께 사는 사람들의 집단.
6,3,'가까스로'의 뜻은?,'가까스로'의 의미를 알려줘?,'가까스로'의 예시가 뭐야?,'가까스로'의 예문을 들어줘,매우 어렵게 힘을 들여.
7,4,'가냘프다'의 뜻은?,'가냘프다'의 의미를 알려줘?,'가냘프다'의 예시가 뭐야?,'가냘프다'의 예문을 들어줘,"몸, 팔, 다리 등이 가늘고 연약하다."
8,5,'가누다'의 뜻은?,'가누다'의 의미를 알려줘?,'가누다'의 예시가 뭐야?,'가누다'의 예문을 들어줘,몸이나 몸의 일부를 바르게 하다.
...,...,...,...,...,...,...
4693,1996,'희소식'의 뜻은?,'희소식'의 의미를 알려줘?,'희소식'의 예시가 뭐야?,'희소식'의 예문을 들어줘,기쁜 소식.
4697,1997,'힐끗'의 뜻은?,'힐끗'의 의미를 알려줘?,'힐끗'의 예시가 뭐야?,'힐끗'의 예문을 들어줘,슬쩍 한 번 흘겨보는 모양.
4698,1998,'힘겹다'의 뜻은?,'힘겹다'의 의미를 알려줘?,'힘겹다'의 예시가 뭐야?,'힘겹다'의 예문을 들어줘,힘이 모자라거나 부족하여 어떤 일을 당해 내기 어렵다.
4699,1999,'힘없다'의 뜻은?,'힘없다'의 의미를 알려줘?,'힘없다'의 예시가 뭐야?,'힘없다'의 예문을 들어줘,기운이나 의욕 등이 없다.


In [76]:
# 데이터프레임을 pickle 파일로 저장
import pickle

with open('/content/train_data.pickle', 'wb') as file:
    pickle.dump(sorted_df, file)

### 구글 공유폴더 이용해서 학습하기
- 구글 드라이브에서 공유폴더 바로가기 생성 > 구글 드라이브 마운트 > MyDrive에서 공유 폴더 바로가기 선택 > 현재 경로 공유 폴더 경로로 변경

In [59]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.0 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/Poly-Encoder

/content/drive/MyDrive/Poly-Encoder


In [66]:
import pickle

with open('/content/drive/MyDrive/Poly-Encoder/datasets/train_data.pickle', 'rb') as f:
# with open('/content/drive/MyDrive/Poly-Encoder/datasets/train.pickle', 'rb') as f:
# with open('/content/drive/MyDrive/Poly-Encoder/datasets/train_with_text.pickle', 'rb') as f:
# with open('/content/drive/MyDrive/Poly-Encoder/datasets/corona_with_text.pickle', 'rb') as f:
# with open('/content/drive/MyDrive/Poly-Encoder/datasets/dasan_train_data.pickle', 'rb') as f:
    corona = pickle.load(f)
corona

,index,Q1,Q2,Q3,Q4,answer
3,1,'가공'의 뜻은?,'가공'의 의미를 알려줘?,'가공'의 예시가 뭐야?,'가공'의 예문을 들어줘,기술이나 힘 등을 이용해 원료나 재료를 새로운 제품으로 만듦.
5,2,'가급적'의 뜻은?,'가급적'의 의미를 알려줘?,'가급적'의 예시가 뭐야?,'가급적'의 예문을 들어줘,할 수 있거나 가능한 것.
6,3,'가까스로'의 뜻은?,'가까스로'의 의미를 알려줘?,'가까스로'의 예시가 뭐야?,'가까스로'의 예문을 들어줘,매우 어렵게 힘을 들여.
8,4,'가누다'의 뜻은?,'가누다'의 의미를 알려줘?,'가누다'의 예시가 뭐야?,'가누다'의 예문을 들어줘,몸이나 몸의 일부를 바르게 하다.
10,5,'가늠'의 뜻은?,'가늠'의 의미를 알려줘?,'가늠'의 예시가 뭐야?,'가늠'의 예문을 들어줘,목표나 기준 등에 맞는지 안 맞는지를 살핌. 또는 살피는 목표나 기준.
...,...,...,...,...,...,...
4681,996,'흡연'의 뜻은?,'흡연'의 의미를 알려줘?,'흡연'의 예시가 뭐야?,'흡연'의 예문을 들어줘,담배를 피움.
4683,997,'흥겹다'의 뜻은?,'흥겹다'의 의미를 알려줘?,'흥겹다'의 예시가 뭐야?,'흥겹다'의 예문을 들어줘,흥이 나서 기분이 좋고 즐겁다.
4688,998,'흩날리다'의 뜻은?,'흩날리다'의 의미를 알려줘?,'흩날리다'의 예시가 뭐야?,'흩날리다'의 예문을 들어줘,흩어져 날리다. 또는 그렇게 하다.
4693,999,'희소식'의 뜻은?,'희소식'의 의미를 알려줘?,'희소식'의 예시가 뭐야?,'희소식'의 예문을 들어줘,기쁜 소식.


In [ ]:
# train poly_encoder
!python utils/run.py \
--model_type bert \
--bert_model models/bert/ \
--output_dir result/train1 \
--train_dir datasets/ \
--train_file train.pickle \
--valid_file train.pickle \
--use_pretrain \
--architecture poly \
--poly_m 16 \
--train_batch_size 2 \
--eval_batch_size 2 \
--max_contexts_length 256 \
--max_response_length 64 \
--num_train_epochs 1

In [ ]:
%cd /content/drive/MyDrive/Poly-Encoder
# train poly_encoder _ 재학습
!python utils/run.py \
--model_type bert \
--bert_model models/bert/ \
--output_dir models/poly_encoder \
--train_dir datasets/ \
--train_file train.pickle \
--valid_file train.pickle \
--use_pretrain \
--architecture poly \
--poly_m 16 \
--train_batch_size 2 \
--eval_batch_size 2 \
--max_contexts_length 256 \
--max_response_length 64 \
--num_train_epochs 1

In [ ]:
# !python3 /content/Poly-Encoder/utils/train.ipynb -bert_model /your/pretrained/model/dir/content/Poly-Encoder/models/bert/pytorch_model.bin --output_dir /your/ckpt/dir --train_dir /your/data/dir --use_pretrain --architecture poly --poly_m 16

In [57]:
%cd /content/drive/MyDrive/Poly-Encoder

/content/drive/MyDrive/Poly-Encoder


In [63]:
# train cross_encoder
!python utils/run.py \
--model_type bert \
--bert_model models/bert/ \
--output_dir result/train2 \
--train_dir datasets/ \
--train_file train_data.pickle \
--valid_file train_data.pickle \
--use_pretrain \
--architecture cross \
--train_batch_size 2 \
--eval_batch_size 2 \
--max_contexts_length 256 \
--max_response_length 64 \
--num_train_epochs 1

2023-09-11 11:19:34.818733: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Namespace(bert_model='models/bert/', eval=False, model_type='bert', output_dir='result/train2', train_dir='datasets/', train_file='train_data.pickle', valid_file='train_data.pickle', test_file='test.pickle', neg_size=15, use_pretrain=True, architecture='cross', max_contexts_length=256, max_response_length=64, train_batch_size=2, eval_batch_size=2, print_freq=100, poly_m=0, learning_rate=5e-05, weight_decay=0.01, warmup_steps=100, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, seed=12345, gradient_accumulation_steps=1, fp16=False, fp16_opt_level='O1', gpu=0)
Train dir: datasets/
Output dir: result/train2
100% 1000/1000 [00:01<00:00, 590.92it/s]
100% 1000/1000 [00:01<00:00, 604.64it/s]
Loading parameters from models/bert/pytorch_model.bin
Some weights of BertModel were not initialized from the model checkpoint at models/bert/ and are newly initialized:

### test 해보기

In [ ]:
from utils.model_for_inference import Load_Model_Tokenizer
import os
import pickle

import torch

CATEGORY = 'train' # normal, water, corona

poly_dir = '/content/drive/MyDrive/Poly-Encoder/models/poly_encoder'
cross_dir = '/content/drive/MyDrive/Poly-Encoder/models/cross_encoder'
emb_dir = '/content/drive/MyDrive/Poly-Encoder/datasets'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# q1, q2, q3, q4, text, embedding(text에 대한) 저장 pickle / 하지만 현재는 inference에 text와 embedding만 사용
with open(os.path.join(emb_dir, f'{CATEGORY}_with_text.pickle'), 'rb') as f:
    embedding_df = pickle.load(f)

cross_encoder, _ = Load_Model_Tokenizer(cross_dir, model_type='cross') # train폴더에서 pytorch_model.bin으로 이름 변경
poly_encoder, tokenizer = Load_Model_Tokenizer(poly_dir, model_type='poly')# train폴더에서 pytorch_model.bin으로 이름 변경

cross_encoder.to(device)
poly_encoder.to(device)


In [ ]:
# # 코드 셀 <undefined>
# # # %% [code]
from utils.inference import Callcenter
import numpy as np

call_center = Callcenter(poly_encoder=poly_encoder, cross_encoder=cross_encoder,
                        tokenizer=tokenizer, emb_df=embedding_df, device=device, topk=10)

query = "'힘겹다'의 예시"
top_k_cross_scores, top_k_indices= call_center.inference(query)
# poly score가 가장 높은 k개의 답변 값들의 cross score 값과 index들 반환

top_cross_idx = top_k_indices[np.argmax(top_k_cross_scores)]
# k개의 답변들 중 cross score가 가장 높은 idx이용하여 최종 답변 산출
answer = embedding_df['text'].iloc[top_cross_idx]

print(f'질문 : {query}')
print(f'답변 : {answer}')

질문 : '힘겹다'의 예시
답변 : 자기 자신을 낮춤.


In [ ]:
%cd /content/drive/MyDrive/Poly-Encoder

/content/drive/MyDrive/Poly-Encoder


In [ ]:
# infrence
!python utils/text_2_emb.py \
--model_type bert \
--bert_model models/bert \
--text_path /content/drive/MyDrive/Poly-Encoder/answers.txt \
--output_dir /content/drive/MyDrive/Poly-Encoder/datasets/train_with_text.pickle \
--gpu 1

2023-09-11 02:02:35.651609: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Namespace(bert_model='models/bert', text_path='/content/drive/MyDrive/Poly-Encoder/answers.txt', max_response_length=128, output_dir='/content/drive/MyDrive/Poly-Encoder/datasets/train_with_text.pickle', model_type='bert', gpu=1)
Loading parameters from models/bert/pytorch_model.bin
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 42001. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 42

### 원본 코드

In [ ]:
parlai interactive -m transformer/polyencoder \
    -mf zoo:pretrained_transformers/model_poly/model \
    --encode-candidate-vecs true \
    --eval-candidates fixed  \
    --fixed-candidates-path data/models/pretrained_transformers/convai_trainset_cands.txt

SyntaxError: ignored

### 다른 git코드

In [ ]:
!git clone

In [ ]:
python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture poly --poly_m 16 --eval

### 끝

In [ ]:
a = [(197, 336), (196, 334), (195, 335), (197, 334), (196, 334), (197, 336), (198, 333), (196, 333), (197, 332), (197, 331), (198, 332), (198, 332), (198, 331), (199, 329), (199, 329), (198, 329), (197, 329), (197, 330), (197, 330), (197, 331), (196, 331), (196, 331), (195, 331), (195, 331), (195, 331), (195, 331), (195, 331), (194, 332), (195, 331), (194, 331), (194, 332), (195, 332), (194, 332), (194, 332), (194, 332), (194, 332), (194, 332), (194, 331), (194, 331), (193, 332), (193, 331), (193, 331), (192, 331), (193, 333), (191, 330), (191, 329), (191, 329), (191, 329), (191, 329), (191, 329), (190, 329), (190, 329), (190, 330), (189, 330), (189, 331), (189, 331), (188, 331), (188, 331), (188, 331), (188, 331), (188, 332), (188, 332), (187, 333), (187, 333), (186, 333), (186, 333), (184, 334), (184, 333), (183, 334), (183, 334), (182, 334), (181, 334), (180, 334), (180, 334), (180, 334), (180, 334), (180, 334), (181, 334), (181, 332), (181, 332), (182, 331), (181, 331), (181, 331), (181, 330), (181, 331), (180, 331), (181, 330), (181, 331), (180, 330), (180, 331), (181, 331), (181, 331), (181, 331), (180, 330), (182, 331), (182, 331), (183, 332), (183, 332), (183, 332), (183, 331), (183, 332), (183, 331), (184, 332), (184, 333), (185, 335), (185, 332), (185, 332), (185, 332), (185, 332), (184, 332)]
c = len(a)
c

110

[197, 336, 0, 100, 200, 102, 400, 104, 600, 106, 800, 108, 1000, 110, 1200, 112, 1400, 114, 1600, 116, 1800, 118, 2000, 120, 2200, 122, 2400, 124, 2600, 126, 2800, 128, 3000, 130, 3200, 132, 3400, 134, 3600, 136, 3800, 138, 4000, 140, 4200, 142, 4400, 144, 4600, 146, 4800, 148, 5000, 150, 5200, 152, 5400, 154, 5600, 156, 5800, 158, 6000, 160, 6200, 162, 6400, 164, 6600, 166, 6800, 168, 7000, 170, 7200, 172, 7400, 174, 7600, 176, 7800, 178, 8000, 180, 8200, 182, 8400, 184, 8600, 186, 8800, 188, 9000, 190, 9200, 192, 9400, 194, 9600, 196, 9800, 198, 10000, 200, 10200, 202, 10400, 204, 10600, 206, 10800, 208]


[]